In [7]:
import numpy
import urllib
import scipy.optimize
import random
from collections import defaultdict
import nltk
import string
from nltk.stem.porter import *
from sklearn import linear_model
import xgboost as xgb
from sklearn.metrics import accuracy_score ,confusion_matrix
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
import numpy as np
import tensorflow as tf

In [5]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5661142971661548209
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 8028386100
locality {
  bus_id: 2
}
incarnation: 15074268260154570424
physical_device_desc: "device: 0, name: Quadro K5200, pci bus id: 0000:8a:00.0, compute capability: 3.5"
]


In [8]:
config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 56} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

In [9]:
def readjson(f):
  for l in open(f):
    yield eval(l)
data=readjson('News_Category_Dataset.json')
data=list(data)
#h = readjson('News_Category_Dataset12.json')
#h = list(h)
#data = data+h
import nltk
from nltk.corpus import stopwords
sw = set(stopwords.words('english'))

In [10]:
w = [d['headline']+d['authors']+d['short_description'] for d in data]
#w = w + [d['headline']+d['authors']+d['long_description'] for d in h]

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(analyzer = "word", max_features=3000, stop_words=sw)
X = tfidf.fit_transform(w).toarray()  

In [12]:
y = [d['category'] for d in data]
len(y)

124989

In [13]:
ylabel_id = dict(zip(list(set(y)), range(31)) )
dtrain=[ int(ylabel_id[d])  for d in y ]

In [14]:
dtrain=np.array(dtrain)

In [15]:
dtrain.shape

(124989,)

In [16]:
y=keras.utils.to_categorical(dtrain, num_classes=31)

In [17]:
print X.shape
print y.shape

(124989, 3000)
(124989, 31)


In [18]:
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) 

In [18]:
keras.__version__

'2.2.4'

In [19]:
model = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 20-dimensional vectors.
model.add(Dense(1024, activation='relu', input_dim=3000))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
#model.add(Dense(31, activation='softmax'))

model.add(Dense(31))
model.add(Activation(tf.nn.softmax))
#tf.nn.softmax(x, dim=axis)
#model.add(Activation('softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.fit(X_train, y_train,
          epochs=50,
          batch_size=128)
score = model.evaluate(X_test, y_test, batch_size=128)
print score

Epoch 1/50
99991/99991 [==============================] - 8s 76us/step - loss: 2.9070 - acc: 0.2612
Epoch 2/50
99991/99991 [==============================] - 7s 67us/step - loss: 2.6600 - acc: 0.3249
Epoch 3/50
99991/99991 [==============================] - 7s 67us/step - loss: 2.4869 - acc: 0.3622
Epoch 4/50
99991/99991 [==============================] - 7s 66us/step - loss: 2.2791 - acc: 0.4101
Epoch 5/50
99991/99991 [==============================] - 7s 66us/step - loss: 2.0697 - acc: 0.4533
Epoch 6/50
99991/99991 [==============================] - 7s 66us/step - loss: 1.9034 - acc: 0.4941
Epoch 7/50
99991/99991 [==============================] - 7s 66us/step - loss: 1.7730 - acc: 0.5247
Epoch 8/50
99991/99991 [==============================] - 7s 67us/step - loss: 1.6736 - acc: 0.5475
Epoch 9/50
99991/99991 [==============================] - 6s 64us/step - loss: 1.5950 - acc: 0.5655
Epoch 10/50
99991/99991 [==============================] - 7s 66us/step - loss: 1.5275 - acc: 0.5789

In [20]:
print(model.metrics_names)
print score

['loss', 'acc']
[1.611520606047135, 0.6046083686361123]
